In [ ]:
!pip install ultralytics cvzone deep-sort-realtime gdown

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!gdown 'https://drive.google.com/uc?id=14Ol1dOIzf2Ve9ZS0VyB1e0wXk-9y6sFj' -O augmented_20.pt
!gdown 'https://drive.google.com/uc?id=1qgyo5Bwv-lOB0jme9m5tfQvMPyYiN-w7' -O 03.jpg
!gdown 'https://drive.google.com/uc?id=1Yv9hKyOJ8x2OiJqOaOi_cdrJwPB1SfpG' -O sheep_farm.mp4
model = YOLO('/content/augmented_20.pt')
video_path = '/content/sheep_farm.mp4'
output_path = '/content/sheepie_farm.mp4'

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import cvzone
class Tracking():
  def __init__(self, model):
    self.model = model

    self.tracker = DeepSort(
            max_age=3,
            n_init=3,
            nms_max_overlap=0.5,
            max_cosine_distance=0.3,
            nn_budget=None,
            override_track_class=None,
            embedder="mobilenet",
            half=True,
            bgr=True,
            embedder_gpu=True,
            embedder_model_name=None,
            embedder_wts=None,
            polygon=False,
            today=None
        )
    self.model.to('cuda')





  def get_raw_detections(self, img):
    '''
    get raw_detections data for tracker of DeepSORT
    '''
    results = self.model(img)
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            cls = int(box.cls[0])
            conf = float(box.conf[0])

            #only predictions have the confidence score > 0.6 are remained
            if conf > 0.6:
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))
    return detections




  def tracking_and_counting(self, img):

    raw_detections = self.get_raw_detections(img)
    tracks = self.tracker.update_tracks(raw_detections=raw_detections, frame=img)


    for track in tracks:
      track_id = track.track_id
      ltrb = track.to_ltrb()
      x1, y1, x2, y2 = ltrb
      x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
      w, h = x2 - x1, y2 - y1

      if not self.video:
        #handle image-input situation
        self.counting.add(track_id)
      else:
        if not track.is_confirmed(): continue

        #each center point of sheep's head pass the line will increase the statistical result
        passed = ((y2 + y1) / 2) >= ((self.height // 6) * 5)
        if passed:
          self.counting.add(track_id)

      #draw bounding box and id of sheep's head
      cvzone.putTextRect(img, f'ID:{track_id}', (x1+10, y1-10), scale=1, thickness=2, colorT=(0, 0, 0), colorR=(255, 255, 255))
      cvzone.cornerRect(img, (x1, y1, w, h), l=5, rt=1, colorR=(0, 0, 255))

    if self.video:
      cv2.line(img, (0, (self.height // 6) * 5), (self.width, (self.height // 6) * 5), (0, 0, 190), 10)
    cvzone.putTextRect(img, f'Counted:{len(self.counting)}', (50, 50), scale=4, thickness=2, colorT=(0, 0, 0), colorR=(255, 255, 255))
    return img





  def process(self, inp, outp, video=True):
    self.video = video
    #for image only
    if not video:
      #init counting engine
      self.counting = set()
      img = cv2.imread(inp)
      img = self.tracking_and_counting(img)
      cv2.imwrite(outp, img)
      return

    #for video

    #setup video properties
    cap = cv2.VideoCapture(inp)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = (cap.get(cv2.CAP_PROP_FPS) // 3) * 2
    self.width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    self.height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(outp, fourcc, fps, (self.width, self.height))

    #init counting engine
    self.counting = set()

    while cap.isOpened():
      ret, frame = cap.read()
      if not ret: break
      frame = self.tracking(frame)
      out.write(frame)
    out.release()
    cap.release()

In [ ]:
tracking = Tracking(model)
tracking.process('/content/03.jpg', '/content/03_p.jpg', video=False)
tracking.process(video_path, output_path)